### 데이터 분석 및 지도 시각화 Library Load

In [1]:
import pandas as pd
import json
import folium

### 데이터 전처리

In [2]:
df_1 = pd.read_csv("./전력 사용량 14차.csv")
df_1.drop(columns=['Unnamed: 0'], inplace=True)
df_1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년월      2448 non-null   object
 1   연도      2448 non-null   int64 
 2   월       2448 non-null   int64 
 3   시도      2448 non-null   object
 4   사용량     2448 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 95.8+ KB


In [3]:
df_1.tail(20)

,년월,연도,월,시도,사용량
2428,2020_12,2020,12,제주특별자치도,456261
2429,2020_12,2020,12,충청남도,4266597
2430,2020_12,2020,12,충청북도,2434030
2431,2021_12,2021,12,강원도,1573555
2432,2021_12,2021,12,경기도,11803946
2433,2021_12,2021,12,경상남도,3134010
2434,2021_12,2021,12,경상북도,3854201
2435,2021_12,2021,12,광주광역시,761414
2436,2021_12,2021,12,대구광역시,1334812
2437,2021_12,2021,12,대전광역시,814480


In [4]:
year = 2010
month = 1

df_ver = df_1.query('연도 == @year & 월 == @month')

In [5]:
df_1.query('연도 == @year')

,년월,연도,월,시도,사용량
0,2010_1,2010,1,강원도,1480180
1,2010_1,2010,1,경기도,8725482
2,2010_1,2010,1,경상남도,2922775
3,2010_1,2010,1,경상북도,3743904
4,2010_1,2010,1,광주광역시,712924
...,...,...,...,...,...
2256,2010_12,2010,12,전라남도,2356637
2257,2010_12,2010,12,전라북도,1757331
2258,2010_12,2010,12,제주특별자치도,307358
2259,2010_12,2010,12,충청남도,3622961


In [6]:
df_data = df_ver[['시도', '사용량']]
df_data

,시도,사용량
0,강원도,1480180
1,경기도,8725482
2,경상남도,2922775
3,경상북도,3743904
4,광주광역시,712924
5,대구광역시,1331473
6,대전광역시,826373
7,부산광역시,1848492
8,서울특별시,4398183
9,세종특별자치시,0


In [7]:
df_data_test = df_data.to_dict(orient='records')
df_data_test

[{'시도': '강원도', '사용량': 1480180},
 {'시도': '경기도', '사용량': 8725482},
 {'시도': '경상남도', '사용량': 2922775},
 {'시도': '경상북도', '사용량': 3743904},
 {'시도': '광주광역시', '사용량': 712924},
 {'시도': '대구광역시', '사용량': 1331473},
 {'시도': '대전광역시', '사용량': 826373},
 {'시도': '부산광역시', '사용량': 1848492},
 {'시도': '서울특별시', '사용량': 4398183},
 {'시도': '세종특별자치시', '사용량': 0},
 {'시도': '울산광역시', '사용량': 2259608},
 {'시도': '인천광역시', '사용량': 2016429},
 {'시도': '전라남도', '사용량': 2233687},
 {'시도': '전라북도', '사용량': 1707366},
 {'시도': '제주특별자치도', '사용량': 325800},
 {'시도': '충청남도', '사용량': 3408166},
 {'시도': '충청북도', '사용량': 1755905}]

In [8]:
jsonfile = open('Sido.json','r',encoding='utf8').read()     # 시도구분된 지역 json 파일 가져오기
jsondata = json.loads(jsonfile)                             
jsondata_loc = {'type': 'FeatureCollection'}                # Sido.json을 복사해서 일부 특성(Key와 Record) 추가
jsondata_pick = []
for idx in jsondata['features']:
    idx['id'] = idx['properties']['CTP_KOR_NM']             # Sido.
    for i in df_data_test:
        if i['시도'] == idx['properties']['CTP_KOR_NM']:    # Sido.json의 지역과 일치하면 pop이라는 Key를 새로 만들고
            idx['properties']['pop'] = i['사용량']         # 해당 지역의 2021.12. 시점의 물가지수를 입력
    jsondata_pick.append(idx)                               # 추가된 값을 jsondata_pick에 저장
jsondata_loc['features'] = jsondata_pick                    # jsondata_loc의 features 키에 저장

# 지도 생성, location : 시작위치, zoom_start : 시작 시 확대정도
map = folium.Map(location=[36.0068191, 127.6607805],zoom_start=7) 
cho = folium.Choropleth(geo_data=jsondata_loc,              # 색상단계도 생성, geo_data : 참고할 위치정보 데이터
                  data=df_data,                          # data : geo_data와 비교해 맞출 데이터
                  columns=['시도', '사용량'],              # columns : data에서 사용할 열
                  fill_color="YlGnBu",                      # fill_color : 색 
                  key_on='feature.id',                      # key_on : data와 맞출 geo_data의 데이터
                  legend_name='맵').add_to(map)    # legend_name : 범례 제목 입력       
cho.geojson.add_child(                                      # 속성 부여
    folium.features.GeoJsonPopup(['CTP_KOR_NM','pop'],labels=False,style='font-weight : bold')
)   # popup 넣기, geo_data의 properties 키의 값들 중 선택 가능, labels : 열이름 유무, style : 서식
# 전력량을 표시하기 위한 CircleMarker 생성
for _, row in self.df_data.iterrows():
    folium.CircleMarker(location=[row['위도'], row['경도']],
                        radius=row['전력량'], # 전력량 값을 radius로 사용
                        color='gray',
                        fill_color='gray',
                        fill_opacity=0.7,
                        tooltip=row['지역']).add_to(self.pop_map)
map

NameError: name 'self' is not defined